## The forward and backward passes

In [7]:
# import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
import pickle,gzip,math,os,time,shutil,jax,jax.numpy as jnp,matplotlib as mpl, numpy as np
from pathlib import Path
# from torch import tensor
from fastcore.test import test_close
# torch.manual_seed(42)
np.random.seed(42)

mpl.rcParams['image.cmap'] = 'gray'
# torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
jnp.set_printoptions(precision=2, linewidth=125, suppress=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
# x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])
x_train, y_train, x_valid, y_valid = map(jnp.array, [x_train, y_train, x_valid, y_valid])

CUDA backend failed to initialize: Unable to load CUDA. Is it installed? (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Foundations version

### Basic architecture

In [8]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, Array(10, dtype=int32))

In [9]:
# num hidden
nh = 50

In [74]:
# w1 = torch.randn(m,nh)
# b1 = torch.zeros(nh)
# w2 = torch.randn(nh,1)
# b2 = torch.zeros(1)

In [15]:
key = jax.random.PRNGKey(42)

In [16]:
w1 = jax.random.normal(key, (m,nh))
b1 = jnp.zeros(nh)
w2 = jax.random.normal(key, (nh,1))
b2 = jnp.zeros(1)

In [17]:
def lin(x, w, b): return x@w + b

In [18]:
t = lin(x_valid, w1, b1)
t.shape

(10000, 50)

In [19]:
# def relu(x): return x.clamp_min(0.)
def relu(x): return jnp.clip(x, a_min=0.) 

In [20]:
t = relu(t)
t

Array([[ 0.  ,  3.36, 16.69, ..., 13.84,  0.  ,  0.  ],
       [ 7.77, 11.85,  5.74, ...,  2.72,  0.  , 16.09],
       [10.55, 20.68,  8.75, ...,  5.03,  5.69,  0.  ],
       ...,
       [13.19,  0.  ,  9.4 , ...,  4.25,  0.64,  8.74],
       [ 2.07,  7.38,  9.38, ...,  0.  ,  0.06,  1.9 ],
       [ 0.  ,  0.  , 17.91, ..., 10.71,  0.  ,  7.87]], dtype=float32)

In [21]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [22]:
res = model(x_valid)
res.shape

(10000, 1)

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [23]:
res.shape,y_valid.shape

((10000, 1), (10000,))

In [24]:
(res-y_valid).shape

(10000, 10000)

We need to get rid of that trailing (,1), in order to use `mse`.

In [25]:
res[:,0].shape

(10000,)

In [26]:
res.squeeze().shape

(10000,)

In [27]:
(res[:,0]-y_valid).shape

(10000,)

In [31]:
# y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

(50000, 1)

In [39]:
def mse(output, targ): return jax.lax.pow((output[:,0]-targ), 2).mean()

In [40]:
mse(preds, y_train)

Array(2633.46, dtype=float32)

### Gradients and backward pass

In [42]:
# !pip install sympy 

  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [43]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [44]:
diff(3*x**2+9, x)

6*x

In [75]:
# def lin_grad(inp, out, w, b):
#     # grad of matmul with respect to input
#     inp.g = out.g @ w.t()
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     b.g = out.g.sum(0)

def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ jnp.transpose(w)
    w.g = (jnp.expand_dims(inp, -1) * jnp.expand_dims(out.g, 1)).sum(0)
    b.g = out.g.sum(0)

In [57]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    # loss = diff.pow(2).mean()
    loss = jax.lax.pow(diff, 2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    # l1.g = (l1>0).float() * l2.g
    l1.g = (l1>0) * l2.g
    lin_grad(inp, l1, w1, b1)

In [76]:
forward_and_backward(x_train, y_train)

In [77]:
# Save for testing against later
# def get_grad(x): return x.g.clone()
def get_grad(x): return x.g.copy()

chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

We cheat a little bit and use PyTorch autograd to check our results.

In [78]:
# def mkgrad(x): return x.clone().requires_grad_(True)
def mkgrad(x): return jax.grad(x.copy())

ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

TypeError: Expected a callable value, got [[ 0.27 -0.95 -1.89 ... -1.47  1.05 -0.3 ]
 [-0.94 -0.7  -0.82 ...  0.24 -0.89 -1.46]
 [ 0.51  0.29  1.41 ...  1.07 -0.72  0.04]
 ...
 [-0.39 -0.51  0.01 ...  0.19 -0.95  0.64]
 [ 0.57  0.09  1.72 ... -0.3  -0.04 -1.41]
 [-1.09 -0.61 -0.85 ... -0.52 -0.05 -0.25]]

In [64]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [65]:
loss = forward(xt2, y_train)
loss.backward()

NameError: name 'xt2' is not defined

In [66]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

NameError: name 'ptgrads' is not defined

## Refactor model

### Layers as classes

In [68]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        # self.out = inp.clamp_min(0.)
        self.out = jnp.clip(inp, a_min=0.)
        return self.out
    
    # def backward(self): self.inp.g = (self.inp>0).float() * self.out.g
    def backward(self): self.inp.g = (self.inp>0) * self.out.g

In [70]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    # def backward(self):
    #     self.inp.g = self.out.g @ self.w.t()
    #     self.w.g = self.inp.t() @ self.out.g
    #     self.b.g = self.out.g.sum(0)

    def backward(self):
        self.inp.g = self.out.g @ jnp.transpose(self.w)
        self.w.g = jnp.transpose(self.inp) @ self.out.g
        self.b.g = self.out.g.sum(0)

In [73]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        # self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]
        self.inp.g = 2. * jnp.expand_dims(self.inp.squeeze() - self.targ, -1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [ ]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [ ]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])